In [78]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Conv1D, LSTM, Dense, Flatten, Reshape


In [79]:
# Load the dataset
df = pd.read_csv('Data1_Columbia.csv')

In [80]:
df.head()

,T2M,T2MWET,T2M_MAX,T2M_MIN,T2MDEW,RH2M,WS2M_MAX,WS2M_MIN,WD10M,PS,Rainfall,Unnamed: 11
0,26.12,24.44,26.70,25.63,22.75,81.56,4.15,0.57,56.50,101.19,8.20,NaN
1,26.96,25.01,27.42,26.15,23.05,79.19,5.38,4.21,47.94,101.17,0.72,NaN
2,26.15,23.98,26.62,25.89,21.82,77.00,2.18,0.19,158.00,101.09,5.68,NaN
3,26.02,23.74,26.38,25.80,21.47,75.94,2.73,0.68,87.69,101.16,4.25,NaN
4,26.12,24.44,26.70,25.63,22.75,81.56,4.15,0.57,56.50,101.19,8.20,NaN


In [99]:
df = df.drop(Unnamed, axis=12)

NameError: name 'Unnamed' is not defined

In [81]:
df.shape

(28188, 12)

In [82]:
# Preprocessing the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df) 

In [83]:
# Split the data into training and testing sets
train_size = int(len(df) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

In [84]:
# Define the window size
window_size = 5

In [85]:
# Create input and target data
def create_dataset(dataset, window_size):
    data_X, data_Y = [], []
    for i in range(len(dataset) - window_size):
        data_X.append(dataset[i:i + window_size, :-1])
        data_Y.append(dataset[i + window_size, -1])  # Target variable (last column)
    return np.array(data_X), np.array(data_Y)

In [86]:
train_X, train_Y = create_dataset(train_data, window_size)
test_X, test_Y = create_dataset(test_data, window_size)

In [87]:
train_X = train_X.reshape((train_X.shape[0], train_X.shape[1], train_X.shape[2], 1))
test_X = test_X.reshape((test_X.shape[0], test_X.shape[1], test_X.shape[2], 1))


In [88]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(window_size, train_X.shape[2])))
model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
model.add(Flatten())
model.add(Reshape((window_size, -1)))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mse')


In [89]:
# Train the model
model.fit(train_X, train_Y, epochs=10, batch_size=32, verbose=1)


Epoch 1/10
705/705 [==============================] - 12s 8ms/step - loss: nan
Epoch 2/10
705/705 [==============================] - 6s 9ms/step - loss: nan
Epoch 3/10
705/705 [==============================] - 6s 9ms/step - loss: nan
Epoch 4/10
705/705 [==============================] - 6s 9ms/step - loss: nan
Epoch 5/10
705/705 [==============================] - 6s 8ms/step - loss: nan
Epoch 6/10
705/705 [==============================] - 6s 9ms/step - loss: nan
Epoch 7/10
705/705 [==============================] - 5s 8ms/step - loss: nan
Epoch 8/10
705/705 [==============================] - 5s 7ms/step - loss: nan
Epoch 9/10
705/705 [==============================] - 5s 7ms/step - loss: nan
Epoch 10/10
705/705 [==============================] - 5s 7ms/step - loss: nan


In [90]:
# Make predictions on the test data
predicted_data = model.predict(test_X)

177/177 [==============================] - 2s 3ms/step


In [98]:
from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Fit the scaler on your training data
scaler.fit(train_data)

# Inverse scale the predicted data
predicted_data = scaler.inverse_transform(predicted_data)

# Print the predicted values
print(predicted_data)


c:\Users\deepr\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:473: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
c:\Users\deepr\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:474: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


ValueError: operands could not be broadcast together with shapes (5633,11) (12,) (5633,11) 

In [94]:
# Assuming you have trained your model and have the test data as test_X

# Make predictions on the test data
predictions = model.predict(test_X)

# Print the predictions
print(predictions*100)


177/177 [==============================] - 1s 3ms/step
[[nan]
 [nan]
 [nan]
 ...
 [nan]
 [nan]
 [nan]]
